#   Cost Prediction - Medical Insurance
#### Debartha Paul

##  Introduction
We'll be doing a Multiple Linear Regression for predicting the cost for Medical Insurance. The dataset is available at [Kaggle](https://www.kaggle.com/datasets/mirichoi0218/insurance).

##  Loading libraries
We'll use seven libraries in this work:
-   `CSV.jl`:   For reading the dataset
-   `DataFrames.jl` and `CategoricalArrays.jl`:    For visualising and cleaning data
-   `Random.jl`, `StableRNGs.jl` and `Distributions.jl`: For splitting data into train and test
-   `GLM.jl`:   For model fitting and prediction

In [1]:
using Pkg
Pkg.add(["CSV", "DataFrames", "CategoricalArrays", "Random",
        "StableRNGs", "Distributions", "GLM"])
using CSV
using DataFrames, CategoricalArrays
using Random, StableRNGs, Distributions
using GLM

    Updating registry at `/srv/julia/pkg/registries/General.toml`
   Resolving package versions...
   Installed StableRNGs ──────── v1.0.0
   Installed ShiftedArrays ───── v2.0.0
   Installed GLM ─────────────── v1.8.0
   Installed StatsModels ─────── v0.6.33
   Installed CategoricalArrays ─ v0.10.8
┌ Warning: The active manifest file at `/home/jovyan/Manifest.toml` has an old format that is being maintained.
│ To update to the new format run `Pkg.upgrade_manifest()` which will upgrade the format without re-resolving.
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.7/Pkg/src/manifest.jl:287
    Updating `~/Project.toml`
  [336ed68f] + CSV v0.8.5
  [324d7699] + CategoricalArrays v0.10.8
  [a93c6f00] + DataFrames v1.2.2
  [31c24e10] + Distributions v0.24.18
  [38e38edf] + GLM v1.8.0
  [860ef19b] + StableRNGs v1.0.0
  [9a3f8284] + Random
    Updating `~/Manifest.toml`
  [324d7699] ↑ CategoricalArrays v0.10.0 ⇒ v0.10.8
  [38e38edf] + GLM v1.8.0
  [1277b4bf

##  Loading and cleaning data
We first load the dataset and see it's dimension

In [2]:
df = CSV.read(raw"insurance.csv", DataFrame)
size(df)

(1338, 7)

Then we see it's first few rows

In [3]:
first(df, 6)

,age,sex,bmi,children,smoker,region,charges
,Int64,String,Float64,Int64,String,String,Float64
1,19,female,27.9,0,yes,southwest,16884.9
2,18,male,33.77,1,no,southeast,1725.55
3,28,male,33.0,3,no,southeast,4449.46
4,33,male,22.705,0,no,northwest,21984.5
5,32,male,28.88,0,no,northwest,3866.86
6,31,female,25.74,0,no,southeast,3756.62


We note that not all of the covariates are quantitative. Some are categorical:
-   `sex`
-   `smoker`
-   `region`

We convert these values to categorical type.

In [4]:
df[!, :sex] = CategoricalArray(df[!, :sex]);
df[!, :smoker] = CategoricalArray(df[!, :smoker]);
df[!, :region] = CategoricalArray(df[!, :region]);

An important thing to note is the baseline selected for each of the categorical values. Thus, we see the levels of the categories.

In [5]:
levels(df[!, :sex])

2-element Vector{String}:
 "female"
 "male"

In [6]:
levels(df[!, :smoker])

2-element Vector{String}:
 "no"
 "yes"

In [7]:
levels(df[!, :region])

4-element Vector{String}:
 "northeast"
 "northwest"
 "southeast"
 "southwest"

Thus the baseline for `sex`, `smoker` and `region` are "female", "no" and "northeast" respectively

##  Training and Test set
The next job will be to subset the data into training and testing set. We set aside 20% of the data as test set to test our model, which we create using the remaining 80% of the data.

In [8]:
rng = StableRNG(2023)
index = rand(rng, Bernoulli(0.8), nrow(df));
train = df[index .== 1, : ];
test = df[index .!= 1, :];

##  Creating the model
We finally create our model using the training set

In [9]:
model = glm(@formula(charges ~ -1 + age + sex + bmi + children + smoker + region),
            train, Normal()
            )

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Normal{Float64}, IdentityLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

charges ~ 0 + age + sex + bmi + children + smoker + region

Coefficients:
────────────────────────────────────────────────────────────────────────────────────
                        Coef.  Std. Error       z  Pr(>|z|)   Lower 95%    Upper 95%
────────────────────────────────────────────────────────────────────────────────────
age                   265.742     13.6073   19.53    <1e-84     239.072     292.412
sex: female        -12421.8     1153.82    -10.77    <1e-26  -14683.3    -10160.4
sex: male          -12754.0     1167.79    -10.92    <1e-27  -15042.8    -10465.1
bmi                   346.191     32.9237   10.51    <1e-25     281.662     410.72
children              566.056    156.284     3.62    0.0003     259.746     872.367
smoker: yes         24129.1      472.532  

The fitted model is of the form:
$$y = \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \beta_5 x_5 + \beta_6 x_6 + \epsilon$$
where:
-   $y$:    The insurance charges of the subject
-   $x_1$:  Age of the subject
-   $x_2$:  Sex of the subject(Categorical)
-   $x_3$:  BMI of the subject
-   $x_4$:  No. of children of the subject
-   $x_5$:  Subject is a smoker?(Categorical)
-   $x_6$:  Region to which the subject belongs
-   $\epsilon$: Error term (assumed to be Gaussian, independently and identically)

Further, each $\beta_j$, $j = 1, 2, \cdots6$, is the coefficient of their respective covariate
and measures the average change in the value of $y$ per unit change in $x_j$.

##  Prediction
We finally predict the outcome in the `test` set and find their MSE

In [10]:
pred = predict(model, DataFrame(age = test[ : , :age], sex = test[ : , :sex],
                bmi = test[ : , :bmi], children = test[ : , :children],
                smoker = test[ : , :smoker], region = test[ : , :region])
                )
mean((pred .- test[ : , :charges]) .^ 2)

3.2636327199220367e7

In [11]:
first(pred, 6)

6-element Vector{Union{Missing, Float64}}:
 12618.597658544715
 12311.764358634951
 14178.455105566769
 32200.896051599975
 -1127.9882303421452
 33936.99179439086